# Check moms data against radial profiles

* load a moms data file
* radial plot of spherical average, compare with RProf 

In order to use the moms data with the ppm.py moms class the moms data written by the code need to be decompressed. See  `How-to-Decompress.md`  in `ppmstar/code/moments_data_reader_ppmsstar2.0` for instructions.

In order to know which of the 32 quantities in the code-written moms data are actually in the 10 decompressed variable slots you have to check the code in the `*DataAnalysis.F` file. The subroutine that actually does all of the work is `getmybqstuff` where the 10 `whatever` slots are set.

In [1]:
%pylab ipympl
import os,sys
from multiprocessing import Pool

ppmpy_dir = '/scratch/f/fherwig/fherwig/repos/PyPPM'
sys.path.insert(0,ppmpy_dir)
from ppmpy import ppm

# set cycling combination of color-blind labels, glyphs, styles
lll= 2*['-', '--', ':', '-.']
markers = ['X','h','<','>','s','^','d','X','p']
random.shuffle(lll)
CB_color_cycle = ['#4daf4a', '#a65628', '#984ea3',
                  '#ff7f00', '#f781bf', '#377eb8',
                  '#999999', '#e41a1c', '#dede00']
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) + cycler('marker',markers[0:8])+cycler('linestyle',lll)))
#rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) +cycler('linestyle',lll)))

# named tuple for using rprofs and momsdata
# moms data are almost always used alongside rprof data and therefore it is recommended 
# to create a convenient dictionary that will hold the rprof and moms instance 
import collections
hydro = collections.namedtuple('hydro', ['moms','rprof'])

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

Populating the interactive namespace from numpy and matplotlib


In [2]:
 print(inspect.getfile(ppm))

/scratch/f/fherwig/fherwig/repos/PyPPM/ppmpy/ppm.py


In [3]:
data_dir = '/scratch/f/fherwig/fherwig'
run_dir = 'M239'
# run_dir="M223-2688-rst-precfrac"
# run_dir = 'M111'
moms_dir = os.path.join(data_dir,run_dir,'moms/myavsbq')
rprof_dir = os.path.join(data_dir,run_dir,'prfs')

At this point check what the decompressed variables are. For M107 these are
```Fortran
       thyng = max(fv(ibq,jbq,kbq),1.0e-12)

       whatever(ibq,jbq,kbq,1) = xc(ibq)
       whatever(ibq,jbq,kbq,2) = rhofux(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,3) = rhofuy(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,4) = rhofuz(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,5) = ut(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,6) = ur(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,7) = vort(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,8) = pf
       whatever(ibq,jbq,kbq,9) = rhof
       whatever(ibq,jbq,kbq,10) = thyng
```
which is the list below (except that we know that in the M100 series runs the vorticity is not calculated correctly, and can't be recovered, and therefore has to be computed from the velocity components).

For M111:
```
# define ismomT 1

#if ismomT
       T9f = 10.**(T9flog10(ibq,jbq,kbq))
       thang = T9f * T90(ibq,jbq,kbq)
#else
       pf = 10.**(pflog10(ibq,jbq,kbq))
       thang = pf * p0(ibq,jbq,kbq)
#endif

       thyng = max(fv(ibq,jbq,kbq),1.0e-12)

       whatever(ibq,jbq,kbq,1) = xc(ibq)
       whatever(ibq,jbq,kbq,2) = rhofux(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,3) = rhofuy(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,4) = rhofuz(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,5) = ut(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,6) = ur(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,7) = vort(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,8) = thang
       whatever(ibq,jbq,kbq,9) = rho
       whatever(ibq,jbq,kbq,10) = thyng
```

In [4]:
# the list of variables that are stored within the momsdata cube.
var_list = ['xc','ux','uy','uz','|ut|','|ur|','|w|','P','rho','fv']
var_list = ['xc','ux','uy','uz','|ut|','|ur|','|w|','T9','rho','fv'] # M111
# var_list = ['xc','ux','uy','uz','|ut|','|ur|','|w|','Pf','rhof','fv'] # M239

## Load data
### Load rprof and moms individually

In [5]:
# load moms instance
dump = 320    # M239
# dump = 1770   # M111
# dump = 1546    # M223-2688-rst-precfrac

moms = ppm.MomsDataSet(moms_dir,init_dump_read=dump,dumps_in_mem=2,var_list=var_list,rprofset=ppm.RprofSet(rprof_dir))

647 rprof files found in '/scratch/f/fherwig/fherwig/M239/prfs/.
Dump numbers range from 0 to 646.
296 .aaa files found in '/scratch/f/fherwig/fherwig/M239/moms/myavsbq/.
Dump numbers range from 311 to 606.


In [6]:
# load rprof data
rp_set = ppm.RprofSet(rprof_dir)
rp = rp_set.get_dump(dump)        # get one dump to access dump data, usually dump 0 should exist, but not for M107
Nx = rp.get('Nx')                 # get grid dimension
print(f'Nx = {Nx}')

647 rprof files found in '/scratch/f/fherwig/fherwig/M239/prfs/.
Dump numbers range from 0 to 646.
Nx = 1152


In [8]:
rp_bavq.get?


Object `rp_bavq.get` not found.


In [10]:
# load rprof
rp_bqav = ppm.RprofSet(rprof_dir, bqav=True, var_list=var_list)
vort = rp_bqav.get('|w|',fname=dump)
rad = rp_bqav.get('xcmax',fname=dump)
# Plot
ifig=109;close(ifig);figure(ifig)
plot(rad[3:], log10(vort[3:]), label='Rprof bqav')

324 rprof files found in '/scratch/f/fherwig/fherwig/M239/prfs/frombqavs/.
Dump numbers range from 311 to 634.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load as dictionary

In [11]:
myrun = hydro(ppm.MomsDataSet(moms_dir,init_dump_read=dump,dumps_in_mem=2,var_list=var_list,rprofset=ppm.RprofSet(rprof_dir)),
              ppm.RprofSet(rprof_dir))

647 rprof files found in '/scratch/f/fherwig/fherwig/M239/prfs/.
Dump numbers range from 0 to 646.
330 .aaa files found in '/scratch/f/fherwig/fherwig/M239/moms/myavsbq/.
Dump numbers range from 311 to 640.
647 rprof files found in '/scratch/f/fherwig/fherwig/M239/prfs/.
Dump numbers range from 0 to 646.


## Basic grid properties

In [13]:
# 3D datacube of the cartesian coordinates at this dump
x,y,z = moms_grid = myrun.moms.get_cgrid()

# it is formatted such that var[z,y,x] refers to the variable at the (z,y,x) coordinates that those 
# indices correspond to. The indexing goes from negative to positive 
moms_ngridpoints = myrun.moms.moms_ngridpoints
print('The unique x,y,z coordinates are:')
print(x[np.random.randint(0,moms_ngridpoints), np.random.randint(0,moms_ngridpoints), :])

# to convert to spherical coordinates
r,theta,phi=myrun.moms.get_sgrid()

The unique x,y,z coordinates are:
[-2690.625 -2671.875 -2653.125 -2634.375 -2615.625 -2596.875 -2578.125
 -2559.375 -2540.625 -2521.875 -2503.125 -2484.375 -2465.625 -2446.875
 -2428.125 -2409.375 -2390.625 -2371.875 -2353.125 -2334.375 -2315.625
 -2296.875 -2278.125 -2259.375 -2240.625 -2221.875 -2203.125 -2184.375
 -2165.625 -2146.875 -2128.125 -2109.375 -2090.625 -2071.875 -2053.125
 -2034.375 -2015.625 -1996.875 -1978.125 -1959.375 -1940.625 -1921.875
 -1903.125 -1884.375 -1865.625 -1846.875 -1828.125 -1809.375 -1790.625
 -1771.875 -1753.125 -1734.375 -1715.625 -1696.875 -1678.125 -1659.375
 -1640.625 -1621.875 -1603.125 -1584.375 -1565.625 -1546.875 -1528.125
 -1509.375 -1490.625 -1471.875 -1453.125 -1434.375 -1415.625 -1396.875
 -1378.125 -1359.375 -1340.625 -1321.875 -1303.125 -1284.375 -1265.625
 -1246.875 -1228.125 -1209.375 -1190.625 -1171.875 -1153.125 -1134.375
 -1115.625 -1096.875 -1078.125 -1059.375 -1040.625 -1021.875 -1003.125
  -984.375  -965.625  -946.875  -928.125  -

In [14]:
2688/4/4

168.0

In [15]:
len(x)

288

In [16]:
670.98213148*4

2683.92852592

## Get decompressed quantities and compare radial profiles with RProf
### Velocity and vorticity magnitude

In [17]:
# dump=2000
# get the cartesian velocity components
ux = myrun.moms.get('ux',fname=dump)
uy = myrun.moms.get('uy',fname=dump)
uz = myrun.moms.get('uz',fname=dump)

# grab |U| on the entire grid, then get radial profile
U = myrun.moms.norm('ux','uy','uz')
U_moms, r_moms = myrun.moms.get_rprof(U,fname=dump)

# RProf data
r_rprof = myrun.rprof.get('R',fname=dump,resolution='l')
U_rprof = myrun.rprof.get('|U|',fname=dump,resolution='l')

# Plot
ifig=1;close(ifig);figure(ifig)
plot(r_moms, U_moms, label='MomsData')
plot(r_rprof, U_rprof, label='Rprof')
legend(); title('|U| velocity magnitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '|U| velocity magnitude')

In [19]:
# This cell could also be used to check radial and tangential velocity components

# vorticity from moms
w_moms, r_moms = myrun.moms.get_rprof('|w|',fname=dump)

# vorticity calculated from velocity components 
# from http://206-12-89-164.cloud.computecanada.ca/csa/hcorem25-paper1/-/blob/master/notebooks/calc_vortmags.py
def get_vel_rprofs_from_moms(star_moms, DUMP, things=['w', 'ur', 'ut'], rad_space=None, workers=1):
    '''                                                                                                 
    Calculates a quantities for Radial Profiles from the MomsDataSet instance                           
    for a given dump. Compatibility with multiprocessesing.                                             
                                                                                                        
    Parameters                                                                                          
    ----------                                                                                          
    star_moms : MomsDataSet                                                                             
        The MomsDataSet instance for which you want the radial profiles.                                
    DUMP : int                                                                                          
        The dump number for which to get radial profile                                                 
    things : list                                                                                       
        A subset of ['w', 'ur', 'ut'], for which to calculate the profiles.                             
    rad_space : float, None                                                                             
        The spacing between the radii for which to calcualte the spherical averages.                    
        If None, will take the moms grid size for the spacing.                                          
        The default is None.                                                                            
    workers : int, optional                                                                             
        Number of workers for multiprocessesing Pool.                                                   
        The default is 1. (Will not use multiprocessesing)                                              
                                                                                                        
    Returns                                                                                             
    -------                                                                                             
    numpy.ndarray                                                                                       
        2D array, first array is radii, the rest are pairs of computed and whatever.                    
    '''

    moms_grid = star_moms.get_cgrid()
    grid_size = abs(moms_grid[0][0][0][1] - moms_grid[0][0][0][0])
    moms_max_rad = max(moms_grid[0][0][0])

    xs = star_moms.get(0, fname=DUMP)
    ux = star_moms.get(1, fname=DUMP)
    uy = star_moms.get(2, fname=DUMP)
    uz = star_moms.get(3, fname=DUMP)

    ur, utheta, uphi = star_moms.get_spherical_components(ux, uy, uz)
    ut = np.sqrt(utheta**2 + uphi**2)

    moms_ut = star_moms.get(4, fname=DUMP)
    moms_ur = star_moms.get(5, fname=DUMP)
    moms_w  = star_moms.get(6, fname=DUMP)

    mX = moms_grid[0]
    mY = moms_grid[1]
    mZ = moms_grid[2]

    dux_dz, dux_dy, dux_dx = np.gradient(ux, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])
    duy_dz, duy_dy, duy_dx = np.gradient(uy, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])
    duz_dz, duz_dy, duz_dx = np.gradient(uz, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])

    curl = (duz_dy - duy_dz, dux_dz - duz_dx, duy_dx - dux_dy)
    vortmag = np.sqrt(curl[0]**2 + curl[1]**2 + curl[2]**2)

    if rad_space == None: rad_space = grid_size
    radii = np.arange(0, moms_max_rad, rad_space)

    global get_avg_velsvort
    def get_avg_velsvort(rad):
#         print("computing radius: {}".format(rad), end='\r')
        npoints = star_moms.sphericalHarmonics_lmax(rad)[-1]
        avg_w = 0; avg_moms_w = 0
        avg_ur = 0; avg_moms_ur = 0
        avg_ut = 0; avg_moms_ut = 0
        if 'ur' in things:
            sphere_ur = star_moms.get_spherical_interpolation(ur, rad, npoints=npoints)
            whatever_ur = star_moms.get_spherical_interpolation(moms_ur, rad, npoints=npoints)
            avg_ur = np.mean(sphere_ur)
            avg_moms_ur = np.mean(whatever_ur)
        if 'ut' in things:
            sphere_ut = star_moms.get_spherical_interpolation(ut, rad, npoints=npoints)
            whatever_ut = star_moms.get_spherical_interpolation(moms_ut, rad, npoints=npoints)
            avg_ut = np.mean(sphere_ut)
            avg_moms_ut = np.mean(whatever_ut)
        if 'w' in things:
            sphere_w = star_moms.get_spherical_interpolation(vortmag, rad, npoints=npoints)
            whatever_w = star_moms.get_spherical_interpolation(moms_w, rad, npoints=npoints)
            avg_w = np.mean(sphere_w)
            avg_moms_w = np.mean(whatever_w)
        return [rad, avg_w, avg_moms_w, avg_ur, avg_moms_ur, avg_ut, avg_moms_ut]

    if workers <= 1:
        return np.array(list(map(get_avg_velsvort, radii))).T

    pol = Pool(workers)
    results = pol.map(get_avg_velsvort, radii)
    return np.array(results).T

results = get_vel_rprofs_from_moms(myrun.moms, dump, things=['w'], rad_space=20, workers=39)
R_moms_calc,w_calc,w_moms_pool = results[0:3]

In [20]:
# Plot
ifig=1;close(ifig);figure(ifig)
plot(r_moms, w_moms, label='MomsData')
plot(R_moms_calc,w_calc, 'o',label='from vel components')
plot(R_moms_calc,w_moms_pool, 's',label='from moms averaging')
plot(rad[3:], vort[3:], label='Rprof bqav')
legend(); title('vort magnitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'vort magnitude')

For M107 this plot shows that the vorticity moms data (green dashed) is incorrect, an from these data sets the vorticiy must be calculated.

### Density, pressure and FV

The plots below show that in M107 the density and pressure are incorrect a well, but the fractional volume FV is correct.

In [23]:
# density radial profile
rho_moms, r_moms = myrun.moms.get_rprof('rho',fname=dump)
# P_moms, r_moms = myrun.moms.get_rprof('P',fname=dump)
T9_moms, r_moms = myrun.moms.get_rprof('T9',fname=dump)

# FV radial profile
FV_moms, r_moms = myrun.moms.get_rprof('fv',fname=dump)

In [25]:
# get density, FV and R, all low resolution
r_rprof = myrun.rprof.get('R',fname=dump,resolution='l')
rho_rprof = (myrun.rprof.get('Rho0',fname=dump,resolution='l') + 
             myrun.rprof.get('Rho1',fname=dump,resolution='l'))
P_rprof = (myrun.rprof.get('P0',fname=dump,resolution='l') + 
             myrun.rprof.get('P1',fname=dump,resolution='l'))
FV_rprof = myrun.rprof.get('FV',fname=dump,resolution='l')

In [27]:
ifig=2;close(ifig);figure(ifig)
plot(r_moms,rho_moms,label='MomsData')
plot(r_rprof,rho_rprof,'--',label='Rprof')
legend(); title('Density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Density')

In [ ]:
ifig=3;close(ifig);figure(ifig)
plot(r_moms,P_moms,label='MomsData')
plot(r_rprof,P_rprof,label='Rprof')
legend(); title('Pressure')

In [28]:
ifig=4;close(ifig);figure(ifig)
plot(r_moms,FV_moms,label='MomsData')
plot(r_rprof,FV_rprof,'--',label='Rprof')
legend(); title('FV')
yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …